# Redcap Form Status Table – All-in-One​
Consolidating all REDCap forms and their status into a single table

### Import Libraries
Libraries necessary to run

In [1]:
from IPython.display import display, Markdown, Latex
import pandas as pd
from pathlib import Path

### Parametrization
Define parameters to be utilized within the notebook -- hint, they should not be equal to anything!

In [2]:
site: str = "duke"
arm: str = "1y_visit_arm_1"

### Create Data Dictionary
Parse through each site + arm + form, and then create an appropriate data dictionary that can be displayed as a table

In [8]:
def create_dictionary(sites, arms):
    # Create blank dictionary
    subjects = {}
    form_statuses = {}

    # Parse through all sites, arms, and forms
    for site in sites:
        p = Path('../../inventory_by_site/' + site)
        for arm_year in p.iterdir():
            if (arm_year.is_dir() and arm_year.stem in arms):
                for form in arm_year.iterdir():
                    if (form.is_dir() == False):
                        df = pd.read_csv(form)

                        # Create blank dictionary if not already a part, or simply set the count
                        for index, row in df.iterrows():
                            if row['study_id'] not in subjects:
                                form_statuses[row['study_id']] = {
                                    'year': arm_year.stem
                                }
                                subjects[row['study_id']] = {
                                    'year': arm_year.stem
                                }
                            subjects[row['study_id']][form.stem] = int(row['non_nan_count'])
                            form_statuses[row['study_id']][form.stem] = row['status']

    return subjects, form_statuses

### Functions to Style Table
Functions to help with styling, in accompaniment with `.apply()`

In [9]:
def determine_color(v):
    if (type(v) is tuple):
        if (0 in v and ('MISSING' in v or 'EXCLUDED' in v)):
            return('color: #a6a6a6; background-color: #e8e4e4;')
        if (0 in v and 'EMPTY' in v):
            return('font-weight: bold; background-color: #e8e4e4;')
        if (0 not in v and ('MISSING' in v or 'EXCLUDED' in v)):
            return('font-weight: bold; background-color: #e8e4e4; text-decoration: line-through;')
        if (0 not in v and 'PRESENT' in v):
            return('background-color: #b0d48c;')
        
        
def style_row(x, form_statuses):
    # Create array of style as well as dataframe for data
    style = [None] * x.size
    sample_df = pd.DataFrame(data=x)
    
    # Initialize beginning to be clear background
    style[0] = 'background-color: #ffffff;'
    for i in range(1, x.size):
        value = sample_df.iloc[i][0]
        form_name = sample_df.iloc[i].name
        form_status = form_statuses[x.name][form_name]
        style[i] = determine_color((value, form_status))
    return style

### Display Table
Using the subjects dictionary, convert the data into a transposed datafram end display the table

In [10]:
def display_table(subjects, form_statuses):
    # Convert to Pandas Dataframe
    final_df = pd.DataFrame(data=subjects).T
    
    # Add Pandas.style
    s3 = final_df.style.apply(lambda x: style_row(x, form_statuses), axis=1)

    # Display Dashboard
    display(Markdown('## Sample Dashboard'))
    display(Markdown('For ' + site.capitalize() + ' and arm ' + arm))
    display(s3)

### Main Function
Define sites and arms, create subjects dictionary, and display table

In [11]:
def main():
    sites = [site]
    arms = [arm]
    subjects, form_statuses = create_dictionary(sites, arms)
    display_table(subjects, form_statuses)

### Run main
Run the main function and watch it do its magic

In [12]:
main()

## Sample Dashboard

For Duke and arm 1y_visit_arm_1